# CO2 Emissions Prediction
## Fase 1 - ETL from World Bank API

## Objetivo
Extrair indicadores do World Development Indicators (WDI) via API do Banco Mundial e gerar:

- `data/raw/wdi_long.csv` (formato longo: país–indicador–ano–valor)
- `data/processed/wdi_wide.csv` (formato wide estilo Kaggle: 1 linha por país+indicador, colunas por ano)

## Decisões de engenharia
- **Sem falha silenciosa**: respostas inesperadas da API geram erro explicativo.
- **Cache por indicador**: salva em `data/raw/indicators/{INDICATOR}.csv`.
- **Robustez**: retries/backoff + timeouts adequados + pausa curta entre páginas.

In [1]:
# Importando bibliotecas necessárias 
import pandas as pd
import json
import logging
import time
import requests
from pathlib import Path
from typing import Any, Dict, List, Optional
from IPython.display import display


In [2]:
# Configura o sistema de logs
logging.basicConfig(level=logging.INFO, format="%(asctime)s | %(levelname)s | %(message)s")
logger = logging.getLogger("wdi_etl")

# Cofigura exibição das tabelas
pd.set_option("display.max_columns", 50)
pd.set_option("display.width", 120)

In [3]:
# Cria session com retry/backoff
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

def build_session() -> requests.Session:
    """ Session com retry/backoff para tornar o ETL robusto a:
    - timeouts intermitentes
    - 429/5xx do servidor
    """
    session = requests.Session()

    retry = Retry(
        total=8,
        connect=8,
        read=8,
        backoff_factor=1.2,  # 1.2s, 2.4s, 4.8s...
        status_forcelist=[429, 500, 502, 503, 504],
        allowed_methods=["GET"],
        raise_on_status=False,
        respect_retry_after_header=True,
    )

    adapter = HTTPAdapter(max_retries=retry, pool_connections=10, pool_maxsize=10)
    session.mount("https://", adapter)
    session.mount("http://", adapter)
    return session

SESSION = build_session()

## Configurações do ETL
- INDICATORS: lista dos códigos WDI (fonte oficial)
- YEARS: intervalo 2000–2020
- PATH_LONG/PATH_WIDE: saídas em disco

In [4]:
# Lista de Indicadores
INDICATORS = [
    "EG.ELC.ACCS.ZS",       # Access to electricity (% of population)
    "AG.LND.AGRI.ZS",       # Agricultural land (% of land area)
    "ER.H2O.FWVT.ZS",       # Annual freshwater withdrawals, total (% of internal resources)
    "AG.LND.ARBL.ZS",       # Arable land (% of land area)
    "AG.LND.FRST.ZS",       # Forest area (% of land area)
    "EG.USE.ELEC.KH.PC",    # Electric power consumption (kWh per capita)
    "EG.USE.PCAP.KG.OE",    # Energy use (kg of oil equivalent per capita)
    "EG.ELC.RNEW.ZS",       # Renewable electricity output (% of total electricity output)
    "EG.FEC.RNEW.ZS",       # Renewable energy consumption (% of total final energy consumption)
    "SP.POP.GROW",          # Population growth (annual %)
    "NY.GDP.PCAP.CD",       # GDP per capita (current US$)
    "EN.ATM.CO2E.PC",       # CO2 emissions (metric tons per capita)
]

# Formato e Range de Anos; Base da API 
YEARS = list(range(2000, 2021))
DATE_RANGE = f"{min(YEARS)}:{max(YEARS)}"
BASE = "https://api.worldbank.org/v2"

In [5]:
# Arquitetura de pastas e arquivos gerados pelo ETL
# Caminho para os arquivos gerados
PATH_LONG = Path("data/raw/wdi_long.csv")
PATH_WIDE = Path("data/processed/wdi_wide.csv")

# Criar pastas
Path("data/raw").mkdir(parents=True, exist_ok=True)
Path("data/processed").mkdir(parents=True, exist_ok=True)
logger.info("Pastas data/raw e data/processed prontas.")

2026-02-26 11:46:41,154 | INFO | Pastas data/raw e data/processed prontas.


## Função utilitária: paginação da API 
API do World Bank é paginada.
A função abaixo:
- percorre todas as páginas
- acumula os itens
- retorna uma lista única

In [6]:
# Função para capturar todas as paginas
import time
def wb_get_all_pages(url: str, params: dict, session: requests.Session = SESSION) -> list:
    """Busca todas as páginas de um endpoint do World Bank (API v2), com resiliência a timeouts e respostas lentas."""
    params = dict(params)
    params.setdefault("format", "json")
    params.setdefault("per_page", 2000)  # menor = resposta mais leve e menos timeout

    page = 1
    out = []

    while True:
        params["page"] = page

        # timeout pode ser tupla: (connect_timeout, read_timeout)
        resp = session.get(url, params=params, timeout=(10, 180))
        resp.raise_for_status()

        data = resp.json()

        if not isinstance(data, list) or len(data) < 2 or data[1] is None:
            break

        meta, items = data[0], data[1]
        out.extend(items)

        pages = int(meta.get("pages", 1))
        if page >= pages:
            break

        page += 1

        # “educado” com a API (ajuda estabilidade)
        time.sleep(0.2)

    return out

## Países e filtro de agregados
O catálogo de países será baixado para:
- identificar e remover “Aggregates” (World, regiões, etc.)
- manter apenas países/territórios como unidades de análise

In [7]:
# Baixar países + valid_country_codes
countries_url = f"{BASE}/country"
countries_items = wb_get_all_pages(countries_url, params={"per_page": 20000})
df_countries = pd.json_normalize(countries_items)

# Agregados normalmente aparecem com region.value == "Aggregates"
if "region.value" in df_countries.columns:
    df_countries["is_aggregate"] = df_countries["region.value"].eq("Aggregates")
else:
    df_countries["is_aggregate"] = False  # fallback defensivo

# Define conjunto de códigos de paises válidos
valid_country_codes = set(df_countries.loc[~df_countries["is_aggregate"], "id"].tolist())

# Exibe o total de paises válidos
logger.info("Total entidades (inclui agregados): %s", df_countries.shape[0])
logger.info("Total países/territórios (sem agregados): %s", len(valid_country_codes))

2026-02-26 11:47:54,418 | INFO | Total entidades (inclui agregados): 296
2026-02-26 11:47:54,419 | INFO | Total países/territórios (sem agregados): 217


## Função: download de 1 indicador (formato longo)
Baixa um indicador WDI para todos os países, no intervalo configurado, e devolve:
- Country Code,
- Country Name,
- Indicator Code,
- Indicator Name,
- Year,
- Value.

In [8]:
# Download de indicador (formato longo)

def download_indicator_long(indicator_code: str) -> pd.DataFrame:
    """Baixa um indicador WDI (todos países) para o intervalo DATE_RANGE. Retorna um DataFrame no formato longo:
        (Country, Indicator, Year) -> Value"""
    url = f"{BASE}/country/all/indicator/{indicator_code}"
    items = wb_get_all_pages(url, params={"date": DATE_RANGE})

    rows = []
    for it in items:
        # A API é semi-estruturada; usamos .get com fallback seguro
        ccode = (it.get("country") or {}).get("id")
        cname = (it.get("country") or {}).get("value")
        icode = (it.get("indicator") or {}).get("id")
        iname = (it.get("indicator") or {}).get("value")
        year = it.get("date")
        val = it.get("value")

        if ccode is None or year is None:
            continue

        rows.append({
            "Country Code": ccode,
            "Country Name": cname,
            "Indicator Code": icode,
            "Indicator Name": iname,
            "Year": int(year),
            "Value": val,
        })

    df = pd.DataFrame(rows)

    # Normaliza tipos (Value pode vir None)
    if not df.empty:
        df["Value"] = pd.to_numeric(df["Value"], errors="coerce")
        df["Year"] = df["Year"].astype(int)

    return df

# Construção do dataset LONG (com cache)
- Se data/raw/wdi_long.csv existir → carrega
- Caso contrário → baixa todos os indicadores e salva

In [9]:
# Construir df_long com cache
if PATH_LONG.exists():
    df_long = pd.read_csv(PATH_LONG)
    df_long["Year"] = df_long["Year"].astype(int)
    df_long["Value"] = pd.to_numeric(df_long["Value"], errors="coerce")
    logger.info("Carregado do cache: %s | shape=%s", PATH_LONG, df_long.shape)
else:
    dfs = []
    for i, ind in enumerate(INDICATORS, start=1):
        logger.info("[%s/%s] Baixando indicador: %s", i, len(INDICATORS), ind)
        dfs.append(download_indicator_long(ind))

    df_long = pd.concat(dfs, ignore_index=True)
    df_long = df_long[df_long["Year"].between(min(YEARS), max(YEARS))]

    df_long.to_csv(PATH_LONG, index=False)
    logger.info("Salvo: %s | shape=%s", PATH_LONG, df_long.shape)

2026-02-26 11:48:12,946 | INFO | [1/12] Baixando indicador: EG.ELC.ACCS.ZS
2026-02-26 11:48:25,744 | INFO | [2/12] Baixando indicador: AG.LND.AGRI.ZS
2026-02-26 11:48:28,562 | INFO | [3/12] Baixando indicador: ER.H2O.FWVT.ZS
2026-02-26 11:48:29,104 | INFO | [4/12] Baixando indicador: AG.LND.ARBL.ZS
2026-02-26 11:48:31,892 | INFO | [5/12] Baixando indicador: AG.LND.FRST.ZS
2026-02-26 11:48:34,361 | INFO | [6/12] Baixando indicador: EG.USE.ELEC.KH.PC
2026-02-26 11:48:36,503 | INFO | [7/12] Baixando indicador: EG.USE.PCAP.KG.OE
2026-02-26 11:48:38,961 | INFO | [8/12] Baixando indicador: EG.ELC.RNEW.ZS
2026-02-26 11:48:41,745 | INFO | [9/12] Baixando indicador: EG.FEC.RNEW.ZS
2026-02-26 11:48:44,842 | INFO | [10/12] Baixando indicador: SP.POP.GROW
2026-02-26 11:48:47,865 | INFO | [11/12] Baixando indicador: NY.GDP.PCAP.CD
2026-02-26 11:48:49,843 | INFO | [12/12] Baixando indicador: EN.ATM.CO2E.PC
2026-02-26 11:48:50,567 | INFO | Salvo: data/raw/wdi_long.csv | shape=(55860, 6)


In [10]:
# verificando a estrutura do df_long
df_long.head()

,Country Code,Country Name,Indicator Code,Indicator Name,Year,Value
0,ZH,Africa Eastern and Southern,EG.ELC.ACCS.ZS,Access to electricity (% of population),2020,46.282371
1,ZH,Africa Eastern and Southern,EG.ELC.ACCS.ZS,Access to electricity (% of population),2019,44.390861
2,ZH,Africa Eastern and Southern,EG.ELC.ACCS.ZS,Access to electricity (% of population),2018,43.035073
3,ZH,Africa Eastern and Southern,EG.ELC.ACCS.ZS,Access to electricity (% of population),2017,40.223744
4,ZH,Africa Eastern and Southern,EG.ELC.ACCS.ZS,Access to electricity (% of population),2016,38.859598


## Pré-análise de valores ausentes (missing) — `df_longo` (2000–2021)

### Qual a razão para isso?
Antes de transformar o dataset para o formato **wide (Kaggle)**, avalia-se a **qualidade/completude** dos dados para evitar:
- viés por séries incompletas,
- indicadores “muito furados” dominando o tratamento,
- decisões de imputação/remoção feitas no escuro.

> Observação: testamos anos mais recentes e vimos muitos ausentes em **2022–2024**. Por isso, voltamos o recorte para **2000–2021**.

### Mensuração (visão geral)
No recorte **2000–2021**:
- **Shape do dataset:** `55.860` linhas × `6` colunas  
- **Ausentes em `Value`:** `4.349`  
- **Percentual de missing em `Value`:** **7,79%**  
- As demais colunas (`Country Code`, `Country Name`, `Indicator Code`, `Indicator Name`, `Year`) ficaram **sem missing** no recorte analisado.

### Agrupamentos que realizamos
Para entender onde o missing se concentra, medimos o percentual de ausentes em `Value` nos seguintes níveis:

1) **Por indicador** (`Indicator Code`, `Indicator Name`)  
   Objetivo: identificar variáveis com alto missing (potencialmente problemáticas para ML).  
   Destaques:
   - `EG.USE.ELEC.KH.PC` (kWh per capita): **27,48%**
   - `EG.USE.PCAP.KG.OE` (energy use per capita): **25,19%**
   - Demais indicadores ficaram bem abaixo disso (ex.: ~0,38% a ~6%).

2) **Por país/entidade** (`Country Code`, `Country Name`)  
   Objetivo: detectar países/territórios muito incompletos (podem distorcer treino/validação).  
   Exemplos com missing alto:
   - `XY` (Not classified): **100%** (candidato direto a remoção)
   - Alguns territórios pequenos (ex.: Sint Maarten, Monaco, Macao, Kosovo etc.) com missing elevado.

3) **Por ano** (`Year`)  
   Objetivo: verificar se existe “quebra” de cobertura em anos específicos.  
   Resultado: missing ficou relativamente **estável** ao longo do tempo (aprox. **7% a 9%**), sem um único ano “colapsado”.

### Conclusão desta pré-análise
- O missing global (**7,79%**) é **administrável**, mas ele é **concentrado** em alguns indicadores e países/territórios.
- Isso reforça que faz sentido:
  - remover casos claramente não analíticos (ex.: `XY - Not classified`, agregados/regiões),
  - e tratar imputação/estratégia de missing com mais cuidado após estruturar no formato wide.

### Próximo passo sugerido
Depois da limpeza de não analíticos, transformar para o **wide (Kaggle)** e então decidir a regra de tratamento (ex.: interpolação temporal por país+indicador, ou limiar para remover indicadores com missing alto).

In [11]:
# Exibe o shape do df_long
print("shape:", df_long.shape)

# Valores ausentes absolutos e relativos 
# Percentual de valores ausentes em relação ao total de linhas
missing_abs = df_long.isna().sum()
missing_pct = (df_long.isna().mean() * 100).round(2)

# Monta tabela com valores ausente absolutos e relaticos
df_missing = (
    pd.DataFrame({"Ausentes (abs)": missing_abs, "Ausentes (%)": missing_pct})
    .sort_values(by=["Ausentes (abs)", "Ausentes (%)"], ascending=False)
)

# Exibe tabela de valores ausentes
display(df_missing)

shape: (55860, 6)


,Ausentes (abs),Ausentes (%)
Value,4349,7.79
Country Code,0,0.00
Country Name,0,0.00
Indicator Code,0,0.00
Indicator Name,0,0.00
Year,0,0.00


In [12]:
# Valores ausentes absolutos e relativos 
# Percentual de valores ausentes por indicador
missing_by_indicator = (
    df_long.groupby(["Indicator Code", "Indicator Name"])["Value"]
    .apply(lambda s: s.isna().mean() * 100)
    .round(2)
    .sort_values(ascending=False)
)

missing_by_indicator.head()

Indicator Code     Indicator Name                                              
EG.USE.ELEC.KH.PC  Electric power consumption (kWh per capita)                     27.48
EG.USE.PCAP.KG.OE  Energy use (kg of oil equivalent per capita)                    25.19
EG.ELC.RNEW.ZS     Renewable electricity output (% of total electricity output)     6.00
AG.LND.ARBL.ZS     Arable land (% of land area)                                     5.16
AG.LND.AGRI.ZS     Agricultural land (% of land area)                               3.65
Name: Value, dtype: float64

In [13]:
# Valores ausentes absolutos e relativos 
# Percentual de valores ausentes por pais
missing_by_country = (
    df_long.groupby(["Country Code", "Country Name"])["Value"]
    .apply(lambda s: s.isna().mean() * 100)
    .round(2)
    .sort_values(ascending=False)
)

missing_by_country.head()

Country Code  Country Name             
XY            Not classified               100.00
MF            St. Martin (French part)      63.81
SX            Sint Maarten (Dutch part)     60.95
MO            Macao SAR, China              60.00
MC            Monaco                        60.00
Name: Value, dtype: float64

In [14]:
# Valores ausentes absolutos e relativos 
# Percentual de valores ausentes por ano
missing_by_year = (
    df_long.groupby("Year")["Value"]
    .apply(lambda s: s.isna().mean() * 100)
    .round(2)
    .sort_index()
)

missing_by_year.head()

Year
2000    8.98
2001    9.02
2002    8.68
2003    8.65
2004    7.48
Name: Value, dtype: float64

## Limpeza inicial (pré-Kaggle wide): remoção de registros não analíticos

### Objetivo
Antes de transformar o dataset para o **formato wide (estilo Kaggle)**, fizemos uma limpeza para garantir que o conjunto represente **países/territórios reais** e evitar “linhas espúrias” que distorcem análise e modelagem.

### Ações executadas
1. **Padronização de tipos**
   - `Country Code` e `Indicator Code` como texto  
   - `Year` como inteiro  
   - `Value` convertido para numérico (`NaN` quando impossível)

2. **Remoção de “Not classified”**
   - Excluímos registros com `Country Code = "XY"` e/ou `Country Name` contendo “Not classified”, pois são categorias sem uso analítico para comparação entre países.

3. **Remoção de agregados (regiões/agrupamentos)**
   - Excluímos entidades agregadas do World Bank (por exemplo, **regiões** como “Africa Eastern and Southern”), usando o catálogo oficial do endpoint `/country` e filtrando onde `region.value == "Aggregates"`.
   - Resultado: mantemos apenas **países/territórios**, que são a unidade correta para o projeto.

### Por que isso importa para o ML?
- Evita que **médias/regiões** “mascarem” padrões de países.
- Reduz viés e ruído em comparações e imputações futuras.
- Garante que o dataset esteja consistente para o pivot no formato Kaggle (colunas por ano).

### Próximo passo
Com o `df_longo` limpo, vamos **pivotar para wide (Kaggle-like)** e, em seguida, decidir a estratégia de tratamento de valores ausentes (ex.: imputação temporal por país+indicador).

In [15]:
# Limpeza “não analíticos” (Not classified) + padronização básica
# Padronização mínima (evita problemas no pivot)
df_long = df_long.copy()
df_long["Country Code"] = df_long["Country Code"].astype(str)
df_long["Indicator Code"] = df_long["Indicator Code"].astype(str)
df_long["Year"] = pd.to_numeric(df_long["Year"], errors="coerce")
df_long["Value"] = pd.to_numeric(df_long["Value"], errors="coerce")

In [16]:
# Remove linhas com Year inválido (se houver)
df_long = df_long.dropna(subset=["Year"]).copy()
df_long["Year"] = df_long["Year"].astype(int)

# Remover "Not classified" (geralmente Country Code == 'XY')
mask_not_classified = (
    df_long["Country Code"].eq("XY") |
    df_long["Country Name"].fillna("").str.contains("Not classified", case=False, na=False)
)

In [17]:
# Checagem das estruturas
print("Linhas antes:", df_long.shape[0])
print("Removendo Not classified:", int(mask_not_classified.sum()))

df_long = df_long.loc[~mask_not_classified].copy()

print("Linhas depois:", df_long.shape[0])

Linhas antes: 55860
Removendo Not classified: 210
Linhas depois: 55650


In [18]:
# Remover agregados/regiões mantendo países/territórios
countries_url = f"{BASE}/country"
countries_items = wb_get_all_pages(countries_url, params={"per_page": 1000})
df_countries = pd.json_normalize(countries_items)

# Marca agregados
df_countries["is_aggregate"] = df_countries["region.value"].eq("Aggregates")

# ATENÇÃO: usar ISO2 (2 letras) para bater com df_long["Country Code"]
valid_country_codes = set(
    df_countries.loc[~df_countries["is_aggregate"], "iso2Code"]
    .dropna()
    .astype(str)
    .tolist()
)

# Limpeza de códigos inválidos
valid_country_codes.discard("")
valid_country_codes.discard("NA")

before = df_long.shape[0]
df_long = df_long[df_long["Country Code"].isin(valid_country_codes)].copy()
after = df_long.shape[0]

print("Linhas removidas (agregados):", before - after)
print("Linhas após remoção:", after)

# Diagnóstico rápido (para confirmar que agora há interseção)
codes_long = set(df_long["Country Code"].unique())
print("Códigos válidos (iso2) no catálogo:", len(valid_country_codes))
print("Códigos presentes no df_long:", len(codes_long))
print("Interseção:", len(codes_long.intersection(valid_country_codes)))

Linhas removidas (agregados): 10290
Linhas após remoção: 45360
Códigos válidos (iso2) no catálogo: 216
Códigos presentes no df_long: 216
Interseção: 216


In [19]:
df_long.shape

(45360, 6)

In [20]:
df_long.head()

,Country Code,Country Name,Indicator Code,Indicator Name,Year,Value
1029,AF,Afghanistan,EG.ELC.ACCS.ZS,Access to electricity (% of population),2020,97.7
1030,AF,Afghanistan,EG.ELC.ACCS.ZS,Access to electricity (% of population),2019,97.7
1031,AF,Afghanistan,EG.ELC.ACCS.ZS,Access to electricity (% of population),2018,93.4
1032,AF,Afghanistan,EG.ELC.ACCS.ZS,Access to electricity (% of population),2017,97.7
1033,AF,Afghanistan,EG.ELC.ACCS.ZS,Access to electricity (% of population),2016,97.7


In [21]:
# Checagens rápidas de integridade (antes do wide)
print("shape:", df_long.shape)
print("Ano min/max:", df_long["Year"].min(), df_long["Year"].max())
print("Missing % em Value:", round(df_long["Value"].isna().mean() * 100, 2))

# Duplicatas na chave (Country, Indicator, Year)
dups = df_long.duplicated(subset=["Country Code", "Indicator Code", "Year"]).sum()
print("Duplicatas (Country, Indicator, Year):", int(dups))

shape: (45360, 6)
Ano min/max: 2000 2020
Missing % em Value: 8.77
Duplicatas (Country, Indicator, Year): 0


In [22]:
# Transformar para wide (estilo Kaggle) e salvar

# pivot_table cria colunas por ano; aggfunc="first" evita erro caso existam duplicatas na chave (país, indicador, ano)
df_wide = (
    df_long.pivot_table(
        index=["Country Code", "Country Name", "Indicator Code", "Indicator Name"],
        columns="Year",
        values="Value",
        aggfunc="first",
    )
    .reset_index()
)

# Reorganiza as colunas: mantém metadados fixos e ordena os anos em ordem crescente
base_cols = ["Country Code", "Country Name", "Indicator Code", "Indicator Name"]
year_cols = sorted([c for c in df_wide.columns if isinstance(c, int)])
df_wide = df_wide[base_cols + year_cols]

# Grava o df_wide em csv
Path("data/processed").mkdir(parents=True, exist_ok=True)
df_wide.to_csv("data/processed/wdi_wide.csv", index=False)

# Exibe o shape do df_wide
print("df_wide shape:", df_wide.shape)
df_wide.head()

df_wide shape: (2015, 25)


Year,Country Code,Country Name,Indicator Code,Indicator Name,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,AD,Andorra,AG.LND.AGRI.ZS,Agricultural land (% of land area),48.936170,48.404255,48.617021,48.638298,48.531915,46.382979,46.340426,44.191489,44.170213,44.191489,42.063830,42.063830,39.914894,40.021277,40.000000,40.021277,40.042553,40.042553,40.072340,40.000213,39.951702
1,AD,Andorra,AG.LND.ARBL.ZS,Arable land (% of land area),2.127660,1.595745,1.808511,1.829787,1.723404,1.702128,1.659574,1.638298,1.617021,1.638298,1.638298,1.638298,1.617021,1.723404,1.702128,1.723404,1.744681,1.744681,1.754255,1.683191,1.635957
2,AD,Andorra,AG.LND.FRST.ZS,Forest area (% of land area),34.042553,34.042553,34.042553,34.042553,34.042553,34.042553,34.042553,34.042553,34.042553,34.042553,34.042553,34.042553,34.042553,34.042553,34.042553,34.042553,34.042553,34.042553,34.042553,34.042553,34.042553
3,AD,Andorra,EG.ELC.ACCS.ZS,Access to electricity (% of population),100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000
4,AD,Andorra,EG.ELC.RNEW.ZS,Renewable electricity output (% of total elect...,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,99.097111,96.040809,96.391459,96.607078,97.383382


In [23]:
# Validar anos, duplicatas e cobertura (rápido)
# 1) Ver quais colunas de ano existem de fato
base_cols = ["Country Code", "Country Name", "Indicator Code", "Indicator Name"]
year_cols = [c for c in df_wide.columns if c not in base_cols]
year_cols_sorted = sorted(year_cols)

print("Primeiro ano:", year_cols_sorted[0], "| Último ano:", year_cols_sorted[-1])
print("Qtd anos:", len(year_cols_sorted))
print("Qtd países:", df_wide["Country Code"].nunique())
print("Qtd indicadores:", df_wide["Indicator Code"].nunique())

# 2) Duplicatas na chave (não deveria ter)
dups = df_wide.duplicated(subset=base_cols).sum()
print("Duplicatas (Country+Indicator):", int(dups))

# 3) % missing por ano (top 10 piores)
missing_by_year = df_wide[year_cols_sorted].isna().mean().sort_values(ascending=False) * 100
print(missing_by_year.head(10).round(2))

Primeiro ano: 2000 | Último ano: 2020
Qtd anos: 21
Qtd países: 216
Qtd indicadores: 10
Duplicatas (Country+Indicator): 0
Year
2000    3.77
2001    3.77
2002    3.33
2003    3.28
2008    2.68
2009    2.38
2010    2.38
2011    2.18
2004    1.94
2017    1.94
dtype: float64


## Fechamento da fase — Pivot para formato Kaggle + validações rápidas

### 1) Transformação do dataset (long → wide)
Nesta etapa, converti o `df_long` (formato **longo**, com 1 linha por **país–indicador–ano**) para `df_wide` (formato **wide estilo Kaggle**), onde:
- Cada linha representa um **par (Country Code, Indicator Code)**, com seus metadados (`Country Name`, `Indicator Name`)
- Cada coluna de ano (ex.: 2000, 2001, …, 2020) contém o valor de `Value` daquele ano
- Foi usado `aggfunc="first"` no `pivot_table` para evitar falha caso existam duplicidades na chave (país–indicador–ano), mantendo o primeiro valor encontrado

Depois do pivot:
- Reorganizei as colunas para manter **metadados fixos primeiro** e os **anos em ordem crescente**
- Salvei o resultado em `data/processed/wdi_wide.csv`
- Verifiquei o `shape` e visualizei uma amostra com `head()`

### 2) Testes e validações (sanidade do `df_wide`)
Rodei um bloco de validação rápida para confirmar a consistência do dataset wide:

1. **Validação de cobertura temporal**
   - Listei as colunas de ano existentes no `df_wide`
   - Confirmei o **primeiro ano**, **último ano** e a **quantidade de anos** disponíveis

2. **Validação de cardinalidade**
   - Conferi a quantidade de **países distintos** (`Country Code`)
   - Conferi a quantidade de **indicadores distintos** (`Indicator Code`)

3. **Checagem de duplicatas (esperado: 0)**
   - Verifiquei duplicidade na chave: `Country Code + Country Name + Indicator Code + Indicator Name`

4. **Qualidade / completude por ano**
   - Calculei o **percentual de valores ausentes por ano** (colunas de ano)
   - Exibi os **10 anos com maior % de missing** para identificar possíveis “buracos” de cobertura

In [1]:
x = 4+5
print(x)

9
